In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [53]:
df = pd.read_csv("C:/Users/dimas/Desktop/Learning/AI agent/ai_agent_memerea/homeworks/S/S02/S02-hw-dataset.csv")
df

,user_id,age,country,purchases,revenue
0,1,25.0,FR,7,749
1,2,24.0,RU,5,1115
2,3,52.0,FR,7,399
3,4,31.0,RU,6,654
4,5,NaN,DE,6,1296
5,6,120.0,FR,-1,785
6,7,46.0,RU,0,0
7,8,28.0,CN,2,456
8,9,39.0,US,4,980
9,10,24.0,RU,7,511


In [54]:
df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user_id    41 non-null     int64  
 1   age        39 non-null     float64
 2   country    41 non-null     object 
 3   purchases  41 non-null     int64  
 4   revenue    41 non-null     int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 1.7+ KB


,user_id,age,purchases,revenue
count,41.000000,39.000000,41.000000,41.000000
mean,20.243902,36.512821,4.829268,820.048780
std,11.659289,18.304259,2.710189,613.127269
min,1.000000,5.000000,-1.000000,0.000000
25%,10.000000,24.000000,3.000000,432.000000
50%,20.000000,33.000000,5.000000,693.000000
75%,30.000000,45.500000,7.000000,1115.000000
max,40.000000,120.000000,11.000000,2475.000000


In [55]:
df.isna().sum()

user_id      0
age          2
country      0
purchases    0
revenue      0
dtype: int64

In [56]:
df.duplicated().sum()

np.int64(1)

In [57]:
suspicious = df[
    (df['age'] < 12) | (df['age'] > 100) |
    (df['purchases'] < 0) | (df['purchases'] > 1000) |
    (df['revenue'] < 0) |
    ((df['purchases'] > 0) & (df['revenue'] == 0)) |
    ((df['purchases'] == 0) & (df['revenue'] > 0))
]
print(f"Найдено подозрительных записей: {len(suspicious)}")
print(suspicious)

Найдено подозрительных записей: 3
    user_id    age country  purchases  revenue
5         6  120.0      FR         -1      785
10       11   24.0      US          4        0
20       21    5.0      RU          8     1488


В ходе анализа датасета были обнаружены следующие проблемы:
 - 2 пустых значения.
 - 1 дублирующаяся строка
 - 2 user_id, которые содержат в себе ошибку на том или ином этапе записи
 - 2 user_id с подозрительным возрастом (120 и 5 лет)

In [58]:
print("Абсолютная частота стран")
country_counts = df['country'].value_counts()
print(country_counts)
print('Относительная частота стран')
print(df['country'].value_counts(normalize=True) * 100)
country_stats = df.groupby('country').agg({
    'purchases': ['mean', 'sum', 'count'],
    'revenue': ['mean', 'sum'],
    'age': ['mean', 'count']
})
print(country_stats)

df['age_group'] = pd.cut(df['age'], 
                         bins=[0, 25, 35, 50, 100], 
                         labels=['18-25', '26-35', '36-50', '50+'])

print("Распределение по возрастным группам:")
print(df['age_group'].value_counts().sort_index())

age_group_stats = df.groupby('age_group', observed=True).agg({
    'purchases': ['mean', 'sum'],
    'revenue': ['mean', 'sum'],
    'user_id': 'count'
})
age_group_stats.columns = ['_'.join(col) for col in age_group_stats.columns]
print(age_group_stats)

Абсолютная частота стран
country
RU    13
FR    12
US     8
DE     6
CN     2
Name: count, dtype: int64
Относительная частота стран
country
RU    31.707317
FR    29.268293
US    19.512195
DE    14.634146
CN     4.878049
Name: proportion, dtype: float64
        purchases                revenue               age      
             mean sum count         mean    sum       mean count
country                                                         
CN       4.500000   9     2  1054.000000   2108  24.000000     2
DE       6.500000  39     6  1445.500000   8673  34.600000     5
FR       4.250000  51    12   675.916667   8111  46.250000    12
RU       4.769231  62    13   790.076923  10271  30.076923    13
US       4.625000  37     8   557.375000   4459  36.714286     7
Распределение по возрастным группам:
age_group
18-25    12
26-35    11
36-50     9
50+       6
Name: count, dtype: int64
           purchases_mean  purchases_sum  revenue_mean  revenue_sum  \
age_group                          

In [59]:
df['buyer_category'] = pd.cut(df['purchases'], 
                               bins=[-1, 0, 3, 6, 20], 
                               labels=['Неактивные', 'Редкие', 'Средние', 'Активные'])
print(df['buyer_category'].value_counts().sort_index())
buyer_stats = df.groupby('buyer_category', observed=True).agg({
    'revenue': ['mean', 'sum', 'count'],
    'age': 'mean'
})
buyer_stats.columns = ['_'.join(col) for col in buyer_stats.columns]
print(buyer_stats)
cross_stats = df.groupby(['country', 'buyer_category'], observed=True)['revenue'].agg(['mean', 'sum', 'count'])
print(cross_stats)

buyer_category
Неактивные     3
Редкие         9
Средние       16
Активные      12
Name: count, dtype: int64
                revenue_mean  revenue_sum  revenue_count   age_mean
buyer_category                                                     
Неактивные          0.000000            0              3  32.666667
Редкие            440.777778         3967              9  39.666667
Средние           884.750000        14156             16  33.400000
Активные         1226.166667        14714             12  31.636364
                           mean   sum  count
country buyer_category                      
CN      Редкие           456.00   456      1
        Активные        1652.00  1652      1
DE      Редкие           738.00   738      1
        Средние         1034.00  3102      3
        Активные        2416.50  4833      2
FR      Неактивные         0.00     0      1
        Редкие           243.00   729      3
        Средние         1198.00  3594      3
        Активные         750.75  

Россия и Франция доминируют в датасете, составляя 31.7% и 29.3% соответственно, Китай представлен минимально (4.9%). Германия показывает самый высокий средний доход на пользователя (1445.5). По возрастным группам самая молодая категория (18-25 лет) наиболее многочисленна (12 человек) и совершает больше покупок в среднем (5.17).

In [60]:
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 10

In [64]:
plt.figure(figsize=(10, 6))
plt.hist(df['revenue'], bins=15, color='steelblue', edgecolor='black', alpha=0.7)
plt.xlabel('Доход (revenue)', fontsize=12, fontweight='bold')
plt.ylabel('Количество пользователей', fontsize=12, fontweight='bold')
plt.title('Распределение дохода от пользователей', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
mean_revenue = df['revenue'].mean()
median_revenue = df['revenue'].median()
plt.axvline(mean_revenue, color='red', linestyle='--', linewidth=2, label=f'Среднее: {mean_revenue:.0f}')
plt.axvline(median_revenue, color='green', linestyle='--', linewidth=2, label=f'Медиана: {median_revenue:.0f}')
plt.legend()

plt.tight_layout()
plt.savefig('figures/histogram_revenue.png', dpi=300, bbox_inches='tight')
plt.close()

In [65]:
plt.figure(figsize=(10, 6))
df.boxplot(column='purchases', by='country', grid=False, patch_artist=True)
plt.xlabel('Страна', fontsize=12, fontweight='bold')
plt.ylabel('Количество покупок', fontsize=12, fontweight='bold')
plt.title('Распределение количества покупок по странам', fontsize=14, fontweight='bold')
plt.suptitle('')  # Убираем автоматический заголовок pandas

plt.tight_layout()
plt.savefig('figures/boxplot_purchases_by_country.png', dpi=300, bbox_inches='tight')
plt.close()

<Figure size 1000x600 with 0 Axes>

In [66]:
# Удаляем строки с пропусками для корректного отображения
df_clean = df.dropna(subset=['age'])

plt.figure(figsize=(12, 7))
countries = df_clean['country'].unique()
colors = plt.cm.tab10(np.linspace(0, 1, len(countries)))

for i, country in enumerate(countries):
    country_data = df_clean[df_clean['country'] == country]
    plt.scatter(country_data['purchases'], country_data['revenue'], 
                label=country, alpha=0.7, s=100, color=colors[i], edgecolors='black')

plt.xlabel('Количество покупок', fontsize=12, fontweight='bold')
plt.ylabel('Доход (revenue)', fontsize=12, fontweight='bold')
plt.title('Зависимость дохода от количества покупок по странам', fontsize=14, fontweight='bold')
plt.legend(title='Страна', fontsize=10)
plt.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('figures/scatter_purchases_revenue.png', dpi=300, bbox_inches='tight')
print("   ✓ Сохранено: figures/scatter_purchases_revenue.png")
plt.close()

   ✓ Сохранено: figures/scatter_purchases_revenue.png
